![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Tutorial 8

## Confidence Interval and Hypothesis Testing

#### Intervalo de confianza

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

qb = QuantBook()
spy = qb.AddEquity("SPY").Symbol

#get SPY data from August 2010 to the present
start_date = datetime(2010, 8, 1, 0, 0, 0)
end_date = qb.Time
spy_table = qb.History(spy, start_date, end_date, Resolution.Daily)

In [ ]:
spy_total = spy_table[['open','close']]
#calculate log returns
spy_log_return = np.log(spy_total.close).diff().dropna()

In [ ]:
print('Population mean:', np.mean(spy_log_return))

Population mean: 0.0005481046867835749


In [ ]:
print('Population standard deviation:',np.std(spy_log_return))

Population standard deviation: 0.010787058234458887


In [ ]:
# Se pueden revisar las tasas de retorno de los 10 y de los 1000 datos mas recientes y antiguos
print('10 days sample returns:', np.mean(spy_log_return.tail(10)))
print('10 days sample standard deviation:', np.std(spy_log_return.tail(10)))
print('1000 days sample returns:', np.mean(spy_log_return.tail(1000)))
print('1000 days sample standard deviation:', np.std(spy_log_return.tail(1000)))

10 days sample returns: 0.0008129861428401774
10 days sample standard deviation: 0.00835988600245765
1000 days sample returns: 0.0005970258230146666
1000 days sample standard deviation: 0.012931517648941647


####Intervalos de confianza

#####Error estandar de la media

$$
  SE=\frac{\sigma}{\sqrt{n}}
$$

Al utilizar la siguiente intervalo:

$$
    (\mu-1.96 \times SE, \mu+1.96 \times SE)
$$

Lo que se esta diciendo es que se manejara un nivel de confianza del 95%

In [ ]:
# En esta formula se esta diciendo que se manejara un intevalo de confianza del 95%

#apply the formula above to calculate confidence interval
bottom_1 = np.mean(spy_log_return.tail(10))-1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
upper_1 = np.mean(spy_log_return.tail(10))+1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
bottom_2 = np.mean(spy_log_return.tail(1000))-1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
upper_2 = np.mean(spy_log_return.tail(1000))+1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
#print the outcomes
print('10 days 95% confidence inverval:', (bottom_1,upper_1))
print('1000 days 95% confidence inverval:', (bottom_2,upper_2))


10 days 95% confidence inverval: (-0.004368524883596551, 0.0059944971692769055)
1000 days 95% confidence inverval: (-0.00020447794470243076, 0.0013985295907317638)


####Prueba de hipótesis 

In [ ]:
# Se parte de la hipótesis que la media es 0
# Entonces se pone a funcionar los datos con la finalidad de averiguar la media y contrastar con la hipótesis

mean_1000 = np.mean(spy_log_return.tail(1000))
std_1000 = np.std(spy_log_return.tail(1000))
mean_10 = np.mean(spy_log_return.tail(10))
std_10 = np.std(spy_log_return.tail(10))
s = pd.Series([mean_10,std_10,mean_1000,std_1000],index = ['mean_10', 'std_10','mean_1000','std_1000'])
print(s)

mean_10      0.000813
std_10       0.008360
mean_1000    0.000597
std_1000     0.012932
dtype: float64


In [ ]:
# Se contrasta la hipótesis con un intervalo de confianza del 95%

bottom = 0 - 1.64*std_1000/np.sqrt(1000)
upper = 0 + 1.64*std_1000/np.sqrt(1000)
print((bottom, upper))

(-0.0006706460097224693, 0.0006706460097224693)


In [ ]:
# Se contrasta la hipótesis con un intervalo de confianza del 90%

bottom = 0 - 1.96*std_1000/np.sqrt(1000)
upper = 0 + 1.96*std_1000/np.sqrt(1000)
print((bottom, upper))

(-0.0008015037677170973, 0.0008015037677170973)


Para llegar a una conclusión adecuada es necesario hallar el Z valor
$$
  Z = \frac{X-\mu}{\frac{\sigma}{\sqrt{n}}}
$$

In [ ]:
# Aquí se calcula el Z valor

print(np.sqrt(1000)*(mean_1000 - 0)/std_1000)


# El resultado indica que se rechaza la hipotesis nula

1.4599689486697156


In [ ]:
# Se pude acceder directamente a este calculo estadistico (Z valor) mediente la siguiente función

import scipy.stats as st
print((1 - st.norm.cdf(1.9488)))

0.02565965688799665


In [ ]:
# Ahora realizar todos los pasos, pero tomando una muestra de 1200
mean_1200 = np.mean(spy_log_return.tail(1200))
std_1200 = np.std(spy_log_return.tail(1200))
z_score = np.sqrt(1200)*(mean_1200 - 0)/std_1200
print('z-score = ',z_score)
p_value = (1 - st.norm.cdf(z_score))
print('p_value = ',p_value)

z-score =  1.7530712831508348
p_value =  0.039794886034644095


#Fin